# Create Flightmanagement DB

## Get Data
from https://github.com/dangeabunea/pluralsight-mongodb-queries

In [1]:
from google.colab import drive
import os

drive.mount('/content/gdrive', force_remount=True)
dir = os.path.join('gdrive', 'My Drive', 'Eurostat', '05 - Data Science for Big Data')
data_dir = os.path.join(dir, 'data')

Mounted at /content/gdrive


## Start Mongo DB

In [2]:
!apt install mongodb

Reading package lists... Done
Building dependency tree       
Reading state information... Done
mongodb is already the newest version (1:3.6.3-0ubuntu1.4).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.


In [3]:
!service mongodb start

 * Starting database mongodb
   ...done.


In [4]:
!ls

gdrive	sample_data


In [5]:
!cd "gdrive/MyDrive/Eurostat/05 - Data Science for Big Data/data" && ls

aircrafts.json	  crew.json	large.zip		 world192.txt
bible.txt	  E.coli	powers_of_two
bible_word_count  flights.json	powers_of_two_and_three


## Feed Data to DB

In [6]:
!mongoimport --db flightmgmt --collection aircrafts --file  "gdrive/MyDrive/Eurostat/05 - Data Science for Big Data/data/aircrafts.json" --drop

2021-11-11T07:28:25.951+0000	connected to: localhost
2021-11-11T07:28:25.951+0000	dropping: flightmgmt.aircrafts
2021-11-11T07:28:25.998+0000	imported 9 documents


In [7]:
!mongoimport --db flightmgmt --collection crew --file  "gdrive/MyDrive/Eurostat/05 - Data Science for Big Data/data/crew.json" --drop

2021-11-11T07:28:26.075+0000	connected to: localhost
2021-11-11T07:28:26.075+0000	dropping: flightmgmt.crew
2021-11-11T07:28:26.117+0000	imported 4 documents


In [8]:
!mongoimport --db flightmgmt --collection flights --file  "gdrive/MyDrive/Eurostat/05 - Data Science for Big Data/data/flights.json" --drop

2021-11-11T07:28:26.200+0000	connected to: localhost
2021-11-11T07:28:26.201+0000	dropping: flightmgmt.flights
2021-11-11T07:28:26.263+0000	imported 10 documents


# MongoClient

In [9]:
from pymongo import MongoClient
client = MongoClient(host='localhost', port= 27017)
client.list_database_names()

['admin', 'config', 'flightmgmt', 'local']

## Database

In [10]:
db = client.flightmgmt

## Collections

In [11]:
aircrafts = db.aircrafts
crew = db.crew
flights = db.flights

# Projection

In [12]:
flights.find_one({})

{'_id': ObjectId('618cc61a3dabb087e22faa3e'),
 'aircraftCode': 'eede6be6-f716-4e2e-bf81-885f0a16a50c',
 'crew': [{'hoursSlept': 4,
   'name': 'Pierre Cotard',
   'nationality': 'France',
   'position': 'Captain'},
  {'hoursSlept': 6, 'name': 'Amanda Lucas', 'position': 'Attendant'},
  {'hoursSlept': 8,
   'name': 'Joe Stan',
   'nationality': 'UK',
   'position': 'Attendant'}],
 'delayed': False,
 'departure': {'city': 'Paris',
  'code': 'CDG',
  'country': 'France',
  'location': {'coordinates': [2.3, 48.8], 'type': 'Point'},
  'runwayLength': 4215},
 'departureDate': datetime.datetime(2020, 2, 20, 21, 20),
 'destination': {'city': 'Lisabon',
  'code': 'LIS',
  'country': 'Portugal',
  'location': {'coordinates': [-9.1, 38.7], 'type': 'Point'},
  'runwayLength': 3805},
 'distanceKm': 1730,
 'duration': 120,
 'type': 'International'}

In [13]:
flights.find_one({}, {"departure.city": 1, "destination.city":1})

{'_id': ObjectId('618cc61a3dabb087e22faa3e'),
 'departure': {'city': 'Paris'},
 'destination': {'city': 'Lisabon'}}

# Sort

In [14]:
list(flights.find({},{"duration": 1, "departure.city": 1, "destination.city":1}).sort([("duration", -1)]))

[{'_id': ObjectId('618cc61a3dabb087e22faa45'),
  'departure': {'city': 'New York'},
  'destination': {'city': 'Paris'},
  'duration': 645},
 {'_id': ObjectId('618cc61a3dabb087e22faa46'),
  'departure': {'city': 'Munich'},
  'destination': {'city': 'New York'},
  'duration': 600},
 {'_id': ObjectId('618cc61a3dabb087e22faa41'),
  'departure': {'city': 'Istanbul'},
  'destination': {'city': 'Lisabon'},
  'duration': 260},
 {'_id': ObjectId('618cc61a3dabb087e22faa3f'),
  'departure': {'city': 'Paris'},
  'destination': {'city': 'Bucharest'},
  'duration': 150},
 {'_id': ObjectId('618cc61a3dabb087e22faa3e'),
  'departure': {'city': 'Paris'},
  'destination': {'city': 'Lisabon'},
  'duration': 120},
 {'_id': ObjectId('618cc61a3dabb087e22faa47'),
  'departure': {'city': 'Bucharest'},
  'destination': {'city': 'Nice'},
  'duration': 120},
 {'_id': ObjectId('618cc61a3dabb087e22faa43'),
  'departure': {'city': 'Munich'},
  'destination': {'city': 'Nice'},
  'duration': 110},
 {'_id': ObjectId('6

# Pagination

In [15]:
flights.count_documents({})

10

In [16]:
list(flights.find({},{"departure.city": 1, "destination.city":1}).limit(2))

[{'_id': ObjectId('618cc61a3dabb087e22faa3e'),
  'departure': {'city': 'Paris'},
  'destination': {'city': 'Lisabon'}},
 {'_id': ObjectId('618cc61a3dabb087e22faa3f'),
  'departure': {'city': 'Paris'},
  'destination': {'city': 'Bucharest'}}]

In [17]:
list(flights.find({},{"departure.city": 1, "destination.city":1}).skip(2).limit(2))

[{'_id': ObjectId('618cc61a3dabb087e22faa40'),
  'departure': {'city': 'Paris'},
  'destination': {'city': 'Lyon'}},
 {'_id': ObjectId('618cc61a3dabb087e22faa41'),
  'departure': {'city': 'Istanbul'},
  'destination': {'city': 'Lisabon'}}]

# Queries

> {field: {$operator: value}}

Selectors with modifiers: 
- \$eq, \$ne, \$in, \$nin, \$lt, \$lte, \$gt, \$gte, $\regex
- https://docs.mongodb.com/manual/reference/operator/query/
- https://www.w3schools.com/python/python_mongodb_query.asp

## \$eq

In [18]:
flights.find_one({"delayed": False})

{'_id': ObjectId('618cc61a3dabb087e22faa3e'),
 'aircraftCode': 'eede6be6-f716-4e2e-bf81-885f0a16a50c',
 'crew': [{'hoursSlept': 4,
   'name': 'Pierre Cotard',
   'nationality': 'France',
   'position': 'Captain'},
  {'hoursSlept': 6, 'name': 'Amanda Lucas', 'position': 'Attendant'},
  {'hoursSlept': 8,
   'name': 'Joe Stan',
   'nationality': 'UK',
   'position': 'Attendant'}],
 'delayed': False,
 'departure': {'city': 'Paris',
  'code': 'CDG',
  'country': 'France',
  'location': {'coordinates': [2.3, 48.8], 'type': 'Point'},
  'runwayLength': 4215},
 'departureDate': datetime.datetime(2020, 2, 20, 21, 20),
 'destination': {'city': 'Lisabon',
  'code': 'LIS',
  'country': 'Portugal',
  'location': {'coordinates': [-9.1, 38.7], 'type': 'Point'},
  'runwayLength': 3805},
 'distanceKm': 1730,
 'duration': 120,
 'type': 'International'}

In [19]:
flights.find_one({"delayed": {"$eq": False}})

{'_id': ObjectId('618cc61a3dabb087e22faa3e'),
 'aircraftCode': 'eede6be6-f716-4e2e-bf81-885f0a16a50c',
 'crew': [{'hoursSlept': 4,
   'name': 'Pierre Cotard',
   'nationality': 'France',
   'position': 'Captain'},
  {'hoursSlept': 6, 'name': 'Amanda Lucas', 'position': 'Attendant'},
  {'hoursSlept': 8,
   'name': 'Joe Stan',
   'nationality': 'UK',
   'position': 'Attendant'}],
 'delayed': False,
 'departure': {'city': 'Paris',
  'code': 'CDG',
  'country': 'France',
  'location': {'coordinates': [2.3, 48.8], 'type': 'Point'},
  'runwayLength': 4215},
 'departureDate': datetime.datetime(2020, 2, 20, 21, 20),
 'destination': {'city': 'Lisabon',
  'code': 'LIS',
  'country': 'Portugal',
  'location': {'coordinates': [-9.1, 38.7], 'type': 'Point'},
  'runwayLength': 3805},
 'distanceKm': 1730,
 'duration': 120,
 'type': 'International'}

In [20]:
from bson.objectid import ObjectId

list(flights.find({"_id": {"$eq": ObjectId('618ca2f53dabb087e22fa5e1')}}))

[]

## \$ne

In [21]:
flights.find_one({"delayed": {"$eq": False}})

{'_id': ObjectId('618cc61a3dabb087e22faa3e'),
 'aircraftCode': 'eede6be6-f716-4e2e-bf81-885f0a16a50c',
 'crew': [{'hoursSlept': 4,
   'name': 'Pierre Cotard',
   'nationality': 'France',
   'position': 'Captain'},
  {'hoursSlept': 6, 'name': 'Amanda Lucas', 'position': 'Attendant'},
  {'hoursSlept': 8,
   'name': 'Joe Stan',
   'nationality': 'UK',
   'position': 'Attendant'}],
 'delayed': False,
 'departure': {'city': 'Paris',
  'code': 'CDG',
  'country': 'France',
  'location': {'coordinates': [2.3, 48.8], 'type': 'Point'},
  'runwayLength': 4215},
 'departureDate': datetime.datetime(2020, 2, 20, 21, 20),
 'destination': {'city': 'Lisabon',
  'code': 'LIS',
  'country': 'Portugal',
  'location': {'coordinates': [-9.1, 38.7], 'type': 'Point'},
  'runwayLength': 3805},
 'distanceKm': 1730,
 'duration': 120,
 'type': 'International'}

In [22]:
flights.find_one({"destination.city": {"$ne": "Lyon"}})

{'_id': ObjectId('618cc61a3dabb087e22faa3e'),
 'aircraftCode': 'eede6be6-f716-4e2e-bf81-885f0a16a50c',
 'crew': [{'hoursSlept': 4,
   'name': 'Pierre Cotard',
   'nationality': 'France',
   'position': 'Captain'},
  {'hoursSlept': 6, 'name': 'Amanda Lucas', 'position': 'Attendant'},
  {'hoursSlept': 8,
   'name': 'Joe Stan',
   'nationality': 'UK',
   'position': 'Attendant'}],
 'delayed': False,
 'departure': {'city': 'Paris',
  'code': 'CDG',
  'country': 'France',
  'location': {'coordinates': [2.3, 48.8], 'type': 'Point'},
  'runwayLength': 4215},
 'departureDate': datetime.datetime(2020, 2, 20, 21, 20),
 'destination': {'city': 'Lisabon',
  'code': 'LIS',
  'country': 'Portugal',
  'location': {'coordinates': [-9.1, 38.7], 'type': 'Point'},
  'runwayLength': 3805},
 'distanceKm': 1730,
 'duration': 120,
 'type': 'International'}

## \$in and $\nin

In [23]:
list(aircrafts.find())

[{'_id': ObjectId('618cc6193dabb087e22faa23'),
  'capacity': 200,
  'code': 'eede6be6-f716-4e2e-bf81-885f0a16a50c',
  'minRunwayLength': 2500,
  'model': 'Boeing 737-800',
  'range': 5765},
 {'_id': ObjectId('618cc6193dabb087e22faa24'),
  'capacity': 189,
  'code': '1b7ad0de-5836-489b-9791-5a81a51cdb81',
  'minRunwayLength': 1700,
  'model': 'Boeing 737-400',
  'range': 3500},
 {'_id': ObjectId('618cc6193dabb087e22faa25'),
  'capacity': 150,
  'code': '1b7ad0de-5836-489b-9791-5a81a51cdb81',
  'minRunwayLength': 2500,
  'model': 'Airbus A320',
  'range': 6000},
 {'_id': ObjectId('618cc6193dabb087e22faa26'),
  'capacity': 124,
  'code': 'a3faaef2-fe54-4949-928f-be93584da471',
  'minRunwayLength': 2255,
  'model': 'Airbus A319',
  'range': 6900},
 {'_id': ObjectId('618cc6193dabb087e22faa27'),
  'capacity': 215,
  'code': 'eede6be6-f716-4e2e-bf81-885f0a16a50c',
  'minRunwayLength': 2975,
  'model': 'Boeing 737-900',
  'range': 5600},
 {'_id': ObjectId('618cc6193dabb087e22faa28'),
  'capaci

In [24]:
list(aircrafts.find({},{"model": 1}))

[{'_id': ObjectId('618cc6193dabb087e22faa23'), 'model': 'Boeing 737-800'},
 {'_id': ObjectId('618cc6193dabb087e22faa24'), 'model': 'Boeing 737-400'},
 {'_id': ObjectId('618cc6193dabb087e22faa25'), 'model': 'Airbus A320'},
 {'_id': ObjectId('618cc6193dabb087e22faa26'), 'model': 'Airbus A319'},
 {'_id': ObjectId('618cc6193dabb087e22faa27'), 'model': 'Boeing 737-900'},
 {'_id': ObjectId('618cc6193dabb087e22faa28'), 'model': 'Embraer E-175'},
 {'_id': ObjectId('618cc6193dabb087e22faa29'), 'model': 'Boeing 747'},
 {'_id': ObjectId('618cc6193dabb087e22faa2a'), 'model': 'Airbus A350'},
 {'_id': ObjectId('618cc6193dabb087e22faa2b'), 'model': 'ATR 72'}]

In [25]:
list(aircrafts.find({"model": {"$eq": "Boeing 737-800"}},
                    {"model": 1}))

[{'_id': ObjectId('618cc6193dabb087e22faa23'), 'model': 'Boeing 737-800'}]

In [26]:
list(aircrafts.find({"model": {"$in": ["Boeing 737-400", "Boeing 737-800", "Boeing 737-900", "Boeing 747"]}},
                    {"model": 1}))

[{'_id': ObjectId('618cc6193dabb087e22faa23'), 'model': 'Boeing 737-800'},
 {'_id': ObjectId('618cc6193dabb087e22faa24'), 'model': 'Boeing 737-400'},
 {'_id': ObjectId('618cc6193dabb087e22faa27'), 'model': 'Boeing 737-900'},
 {'_id': ObjectId('618cc6193dabb087e22faa29'), 'model': 'Boeing 747'}]

In [27]:
list(aircrafts.find({"model": {"$nin": ["Boeing 737-400", "Boeing 737-800", "Boeing 737-900", "Boeing 747"]}},
                    {"model": 1}))

[{'_id': ObjectId('618cc6193dabb087e22faa25'), 'model': 'Airbus A320'},
 {'_id': ObjectId('618cc6193dabb087e22faa26'), 'model': 'Airbus A319'},
 {'_id': ObjectId('618cc6193dabb087e22faa28'), 'model': 'Embraer E-175'},
 {'_id': ObjectId('618cc6193dabb087e22faa2a'), 'model': 'Airbus A350'},
 {'_id': ObjectId('618cc6193dabb087e22faa2b'), 'model': 'ATR 72'}]

## \$regex
Can only be used to query strings

In [28]:
list(aircrafts.find({"model": {"$regex": "^Boeing"}},
                    {"model": 1}))

[{'_id': ObjectId('618cc6193dabb087e22faa23'), 'model': 'Boeing 737-800'},
 {'_id': ObjectId('618cc6193dabb087e22faa24'), 'model': 'Boeing 737-400'},
 {'_id': ObjectId('618cc6193dabb087e22faa27'), 'model': 'Boeing 737-900'},
 {'_id': ObjectId('618cc6193dabb087e22faa29'), 'model': 'Boeing 747'}]

## \$lt and \$lte

In [29]:
list(flights.find({}, {"departureDate": 1, "_id": 0}))

[{'departureDate': datetime.datetime(2020, 2, 20, 21, 20)},
 {'departureDate': datetime.datetime(2020, 2, 20, 21, 30)},
 {'departureDate': datetime.datetime(2020, 2, 20, 17, 15)},
 {'departureDate': datetime.datetime(2020, 2, 20, 23, 0)},
 {'departureDate': datetime.datetime(2020, 2, 21, 15, 45)},
 {'departureDate': datetime.datetime(2020, 3, 22, 14, 10)},
 {'departureDate': datetime.datetime(2020, 3, 21, 11, 10)},
 {'departureDate': datetime.datetime(2020, 3, 20, 9, 20)},
 {'departureDate': datetime.datetime(2020, 3, 20, 18, 10)},
 {'departureDate': datetime.datetime(2020, 2, 22, 20, 0)}]

In [30]:
import datetime

latest_departure = datetime.datetime(2020, 2, 20, 21, 30, 10, 555)

list(flights.find({"departureDate": {"$lte": latest_departure}},
                  {"departure.city": 1, "destination.city": 1, "departureDate": 1, "_id": 0}))

[{'departure': {'city': 'Paris'},
  'departureDate': datetime.datetime(2020, 2, 20, 21, 20),
  'destination': {'city': 'Lisabon'}},
 {'departure': {'city': 'Paris'},
  'departureDate': datetime.datetime(2020, 2, 20, 21, 30),
  'destination': {'city': 'Bucharest'}},
 {'departure': {'city': 'Paris'},
  'departureDate': datetime.datetime(2020, 2, 20, 17, 15),
  'destination': {'city': 'Lyon'}}]

## \$gt and \$gte

In [31]:
list(flights.find({"distanceKm": {"$gte": 2000}},
                  {"departure.city": 1, "destination.city": 1, "distanceKm": 1}))

[{'_id': ObjectId('618cc61a3dabb087e22faa41'),
  'departure': {'city': 'Istanbul'},
  'destination': {'city': 'Lisabon'},
  'distanceKm': 3200},
 {'_id': ObjectId('618cc61a3dabb087e22faa45'),
  'departure': {'city': 'New York'},
  'destination': {'city': 'Paris'},
  'distanceKm': 5700},
 {'_id': ObjectId('618cc61a3dabb087e22faa46'),
  'departure': {'city': 'Munich'},
  'destination': {'city': 'New York'},
  'distanceKm': 6400}]

## ❓ Exercise

1. Find the crew member with Id 618ca2f43dabb087e22fa5d8

2. Find the destination cities of all the flights that are delayed

3. Find the duration of all intercontinental flights

4. Count the number of Airbus planes in the fleet

5. Find destinations that can be reached since 2020-02-21

6. Find all flights shorter than 3 hours, sort them by increasing duration first and descending distance second

# Geospatial Queries
- \$near
- GeoJSON coordinates:
  - geometry on a sphere
  - WGS84 projection system
  - requires a '2dsphere' index on the geographical field
- Legacy coordinates
  - geometry on a plane
  - requires a '2dindex' index on the geographical field

- Point[Longitude, Latitude]

In [32]:
flights.index_information()

{'_id_': {'key': [('_id', 1)], 'ns': 'flightmgmt.flights', 'v': 2}}

In [33]:
from pymongo import GEOSPHERE

flights.create_index([("departure.location", GEOSPHERE)])
flights.create_index([("destination.location", GEOSPHERE)])

'destination.location_2dsphere'

In [34]:
flights.index_information()

{'_id_': {'key': [('_id', 1)], 'ns': 'flightmgmt.flights', 'v': 2},
 'departure.location_2dsphere': {'2dsphereIndexVersion': 3,
  'key': [('departure.location', '2dsphere')],
  'ns': 'flightmgmt.flights',
  'v': 2},
 'destination.location_2dsphere': {'2dsphereIndexVersion': 3,
  'key': [('destination.location', '2dsphere')],
  'ns': 'flightmgmt.flights',
  'v': 2}}

In [35]:
list(flights.find({"departure.location": {"$near": {
    "$geometry": {
        "type": "Point",
        "coordinates": [2.2, 48.85]
    },
    "$maxDistance": 10000
  }}},
                 {"departure.city": 1, "departure.location": 1}))

[{'_id': ObjectId('618cc61a3dabb087e22faa3e'),
  'departure': {'city': 'Paris',
   'location': {'coordinates': [2.3, 48.8], 'type': 'Point'}}},
 {'_id': ObjectId('618cc61a3dabb087e22faa3f'),
  'departure': {'city': 'Paris',
   'location': {'coordinates': [2.3, 48.8], 'type': 'Point'}}},
 {'_id': ObjectId('618cc61a3dabb087e22faa40'),
  'departure': {'city': 'Paris',
   'location': {'coordinates': [2.3, 48.8], 'type': 'Point'}}}]

# Further Readings
- [Microsoft Ignite - Model and Partition your Data in Azure CosmosDB](https://myignite.microsoft.com/sessions/1410edb6-204d-4c45-adbb-ccb49e8135cf?source=/theme/innovate-anywhere-from-multicloud-to-edge)
- [Microsoft Ignite - Accelerate time to insight with Azure Synapse](https://myignite.microsoft.com/sessions/cb857345-7f4b-459f-b8bf-ab481a02be72?source=/theme/innovate-anywhere-from-multicloud-to-edge)
- [Microsoft Ignite - Deploy IoT solutions with Azure SQL Database](https://myignite.microsoft.com/sessions/8e018c10-cbfe-4754-9299-6b900b8b82ea?source=/theme/innovate-anywhere-from-multicloud-to-edge)

In [36]:
!service mongodb stop

 * Stopping database mongodb
   ...done.
